In [132]:
import pandas as pd
import numpy as np

# Import Prior Data

In [133]:
past_data = pd.read_excel('2011-2019_FDA_NME_Approvals.xlsx')

In [134]:
past_data.columns

Index(['DrugName', 'ActiveIngredient', 'DosageForm', 'ApprovalDate',
       'Indication', 'FirstInClass', 'AcceleratedApproval', 'OrphanDrug',
       'MetPDUFAGoal', 'FastTrack', 'FDABreakthrough', 'FDAPriority',
       'FirstCycleApproval', 'FirstInUS', 'ApplicationDocsURL',
       'ApprovalSponsor', 'FilingDate', 'DrugBankURL', 'Type',
       'Biologic Classification', 'FDASource'],
      dtype='object')

In order to keep code reusable for extracting basic data (DrugName, ActiveIngredient, ApprovalDate, Indication) if the FDA website keeps same format, only years_to_extract should be updated. Check fda_url if link is broken after 2022.

For years prior to 2021 there was a html option to open the 'Advancing Health through Innovation: New Drug Approvals' document (rather than a pdf version) which made it a lot easier to scrape. This doc is used to extract columns F-N in excel sheet. For 2021 we are forced to parse the PDF file.

It was mentioned before that this could seem a bit pointless to do this for single-year data (as opposed to just grabbing it manually) which could be something to keep in mind. It seems that the FDA changes format of their reports and website quite regularly.

Note:
Tried Drugs@FDA api, does not include bool data (columns F-N) as of 4-12-22

# CDER data

## Basic data from main FDA website

In [135]:
years_to_extract = ['2020', '2021']

base_url = "https://www.fda.gov/drugs/new-drugs-fda-cders-new-molecular-entities-and-new-therapeutic-biological-products/novel-drug-approvals-"

In [136]:
extracted_data = pd.DataFrame()

for year in years_to_extract:
    # need to further parse since pandas grabs extra rows for 2020 (row 2)
    raw = pd.read_html(base_url + year)[0]

    mask = pd.notna(pd.to_numeric(raw['No.'], errors='coerce'))
    data = raw[mask].reset_index().drop('index', axis=1)
    
    extracted_data = pd.concat([extracted_data, data])

extracted_data = extracted_data.reset_index().drop('index', axis=1)

# remove extras from approved use column
extracted_data['FDA-approved use on approval date*'] = ([x.split('Drug')[0] for x in 
                                                         extracted_data['FDA-approved use on approval date*']])
extracted_data['FDA-approved use on approval date*'] = ([x.split('Press')[0] for x in 
                                                         extracted_data['FDA-approved use on approval date*']])

# make text uppercase
extracted_data['FDA-approved use on approval date*'] = [x.upper() for x in extracted_data['FDA-approved use on approval date*']]
extracted_data['Drug Name'] = [x.upper() for x in extracted_data['Drug Name']]

In [137]:
extracted_data

,No.,Drug Name,Active Ingredient,Approval Date,FDA-approved use on approval date*
0,53.,GEMTESA,vibegron,12/23/2020,TO TREAT OVERACTIVE BLADDER
1,52.,EBANGA,ansuvimab-zykl,12/21/2020,TO TREAT EBOLA
2,51.,ORGOVYX,relugolix,12/18/2020,TO TREAT ADVANCED PROSTATE CANCER
3,50.,MARGENZA,margetuximab (anti-HER2 mAb,12/16/2020,TO TREAT HER2+ BREAST CANCER
4,49.,KLISYRI,tirbanibulin,12/14/2020,TO TREAT ACTINIC KERATOSIS OF THE FACE OR SCALP
...,...,...,...,...,...
98,5,UKONIQ,umbralisib,2/5/2021,TO TREAT MARGINAL ZONE LYMPHOMA AND FOLLICULAR...
99,4,TEPMETKO,tepotinib,2/3/2021,TO TREAT NON-SMALL CELL LUNG CANCER
100,3,LUPKYNIS,voclosporin,1/22/2021,TO TREAT LUPUS NEPHRITIS
101,2,CABENUVA,cabotegravir and rilpivirine (co-packaged),1/21/2021,TO TREAT HIV


## Boolean Data (columns F-N in 2011-2019_FDA_NME_Approvals.xlsx)

### PARSE BOOL DATA FROM HTML FOR 2020

For 2020 (and prior years), the 'New Drug Therapy Approvals 2020' doc was available either as a html or pdf. <br><br> **As of 4/13/22 the FDA website DID NOT provide a html version for 2021 data, only a PDF.** <br><br> This task is much easier and more reliable to do if the html version is provided, but I will demonstrate how to do both: html for 2020 and pdf for 2021

In [138]:
html_url = 'https://www.fda.gov/drugs/new-drugs-fda-cders-new-molecular-entities-and-new-therapeutic-biological-products/new-drug-therapy-approvals-2020'

In [139]:
bool_2020 = pd.read_html(html_url)[1].replace('X', 1).fillna(0)

b_mask = [True if type(x) == int else False for x in bool_2020['1st U.S.']]
bool_2020 = bool_2020[b_mask].reset_index().drop('index', axis=1)

# uppercase trade name
bool_2020['Trade Name'] = [x.upper() for x in bool_2020['Trade Name']]

### PARSE PDF FOR 2021

Here's where it get's a little weird. In order to extract the data we want for columns F-N, we're going to string match the text-ified PDFs until we find where the drugs in each column category are listed. Category designations are identified by the string match "were: " to indicate where the drugs that fall into each category are listed. This works for 2020 and 2021, but it may not be consistent in later years. See raw PDF files for detail.
<br>
<br>
##### The cleanest way:
PyPDF2 kind of works to extract the text, but sadly it misses precisely the text we are looking for. You must use pdftotext if you want to do it this way.
<br>
<br>
##### The simplest way:
Use subprocess and requests to decode and extract the text in the PDF as a string without actually downloading the file. I recommend pasting the raw output into sublime text for help decoding

#### String matches for each column:
<br><br><br>

|Column Name|Excel Column|String Match after parsing pdf|
|-----------|---------------------|------------|
|FirstInClass|F|' in 2021 that CDER identified as first-in-\r\n class'|
|AcceleratedApproval|G|' Accelerated\r\nCDER identified 14 of the   Approval'|
|OrphanDrug|H|'approved in 2021 with the orphan drug\r\ndesignation'|
|MetPDUFGoal|I|'roved in 2021 on or before their PDUFA goal\r\ndates'|
|FastTrack|J|'s.\r\n\r\n     Drugs designated with Fast Track status'|
|FDABreakthrough|K|'\nDrugs designated with Breakthrough therapy status'|
|FDAPriority|L|'he list below.\r\n\r\nDrugs designated Priority Review'|
|FirstCycleApproval|M|'   Novel drugs approved in 2021 on the first cycle'|
|FirstInUS|N|' drugs of 2021 approved first in the United States'|


<br><br><br>
**There's also a designation for "2021 that used at least one expedited\r\n program", does this fit anywhere??**

In [140]:
from subprocess import Popen, PIPE
import requests
import pdftotext

In [141]:
command = ['pdftotext', '-layout', '-', '-']

p = Popen(command, stdout=PIPE, stdin=PIPE)
r = requests.get('https://www.fda.gov/media/155227/download')

output = p.communicate(input=r.content)[0].decode().replace('      ', '')

In [142]:
# pay close attention to the indexing here!!! 
# you want to make sure that the first element in cat_names matches the first element in cat_vals, 
# or else you risk messing up the entire thing

cat_names = [x[-50:] for x in output.split(' were:')][:-1]
cat_vals = [x[:600].upper() for x in output.split('were:')][1:]

In [143]:
ph_df = pd.DataFrame(columns=['Drug Name'] + cat_names)
drugs_2021 = [x for i, x  in enumerate(extracted_data['Drug Name']) if extracted_data['Approval Date'][i][-4:] == '2021']

for drug in drugs_2021:
    row = [drug]
    for cat in cat_vals:
        if drug in cat:
            row.append(1)
        else:
            row.append(0)
    ph_df.loc[len(ph_df)] = row
    
bool_2021 = ph_df.drop('2021 that used at least one expedited\r\n    program', axis=1)

# make sure column order matches!
bool_2021 = bool_2021.rename(columns=dict(zip(bool_2021.columns, bool_2020.columns)))

### Bool data for 2020 and 2021 is complete here

In [144]:
full_bool = pd.concat([bool_2020, bool_2021])

In [145]:
extracted_data.sort_values(by='Drug Name')

,No.,Drug Name,Active Ingredient,Approval Date,FDA-approved use on approval date*
53,50,ADBRY,tralokinumab-ldrm,12/27/2021,TO TREAT MODERATE-TO-SEVERE ATOPIC DERMATITIS
77,26,ADUHELM,aducanumab-avwa,6/7/2021,TO TREAT ALZHEIMER’S DISEASE
95,8,AMONDYS 45,casimersen,2/25/2021,TO TREAT DUCHENNE MUSCULAR DYSTROPHY
32,21.,ARTESUNATE,artesunate,5/26/2020,TO TREAT SEVERE MALARIA
52,1.,AYVAKIT,avapritinib,1/9/2020,TO TREAT ADULTS WITH UNRESECTABLE OR METASTATI...
...,...,...,...,...,...
89,14,ZEGALOGUE,dasiglucagon,3/22/2021,TO TREAT SEVERE HYPOGLYCEMIA
42,11.,ZEPOSIA,ozanimod,3/25/2020,TO TREAT RELAPSING FORMS OF MULTIPLE SCLEROSIS
29,24.,ZEPZELCA,lurbinectedin,6/15/2020,TO TREAT METASTATIC SMALL CELL LUNG CANCER
10,43.,ZOKINVY,lonafarnib,11/20/2020,TO TREAT RARE CONDITIONS RELATED TO PREMATURE ...


In [146]:
full_bool.sort_values(by='Trade Name')

,Trade Name,First in Class,Orphan,FT,BT,Priority,AA,PDUFA,1st Cycle,1st U.S.
0,ADBRY,1,0,0,0,0,0,1,0,0
24,ADUHELM,1,0,1,0,1,1,1,1,1
42,AMONDYS 45,0,1,1,0,1,1,1,1,1
0,ARTESUNATE,0,1,1,1,1,0,1,1,0
1,AYVAKIT,0,1,1,1,1,0,1,1,1
...,...,...,...,...,...,...,...,...,...,...
36,ZEGALOGUE,0,0,0,0,0,0,1,1,1
50,ZEPOSIA,0,0,0,0,0,0,1,1,1
51,ZEPZELCA,0,1,0,0,1,1,1,1,1
52,ZOKINVY,1,1,0,1,1,0,1,1,1


In [203]:
#  'GA 68 PSMA-11' and 'GALLIUM 68 PSMA-11' are technically the same here, 
# abbreviated version has a hit in the api, so I'll use that version
len(set(np.concatenate([np.array(full_bool['Trade Name']), np.array(extracted_data['Drug Name'])])))

extracted_data = extracted_data.replace('GALLIUM 68 PSMA-11', 'GA 68 PSMA-11')

KeyError: 'Trade Name'

## Rest of Columns

What's left?
<br><br>
DosageForm<br>
ApplicationDocsURL<br>
ApprovalSponsor<br>
FilingDate<br>
DrugBankURL<br>
Type<br>
<br><br>
**GOTTA LOOK AT THE BIOLOGICS AND VACCINES SECTION FOR MORE APPROVALS!**<br>
Biologic Classification<br>
FDASource (CBER/CDER) ----- given since were pulling from there anyway

Looks like for some of these columns we can utilize the drugs@fda api to extract information. Not sure how stable the api is, so may need tweaking in future years

In [148]:
from datetime import datetime

In [149]:
def get_dosage_form(products):
    dosage_form = ''
    for i,x in enumerate(products):
        if i == 0:
            dosage_form+=x['dosage_form']
        else:
            if dosage_form != x['dosage_form']:
                dosage_form+=(', ' + x['dosage_form'])
    return dosage_form

def select_letter(orig_app_doc):
    for x in orig_app_doc:
        if x['type'] == 'Letter':
            return x['url'], x['date']

In [150]:
api_data = pd.DataFrame(columns=['DrugName', 'DosageForm', 'ApplicationDocsURL', 
                                 'ApprovalSponsor', 'FilingDate'])

api_base = 'https://api.fda.gov/drug/drugsfda.json?'

manual_check = []


for drug in extracted_data['Drug Name']:
    
    try:
        api_params = dict(
        search='openfda.brand_name:' + '\"' + drug + '\"',
        limit=1
        )

        pull = requests.get(url=api_base, params=api_params).json()

        # dosage form
        dosage_form = get_dosage_form(pull['results'][0]['products'])
        # application docs url
        orig_sub = ([x for x in pull['results'][0]['submissions'] 
                     if x['submission_type'] == 'ORIG'][0]['application_docs'])
        app_url = select_letter(orig_sub)[0]
        # approval sponsor
        app_spon = pull['results'][0]['sponsor_name']
        # filing date
        filing_date = datetime.strptime(select_letter(orig_sub)[1], '%Y%m%d').strftime('%Y-%m-%d')

        row = [drug, dosage_form, app_url, app_spon, filing_date]
        api_data.loc[len(api_data)] = row
    
    except:
        print(drug + " didn't work, check manually")
        manual_check.append(drug)

GALLIUM 68 PSMA-11 didn't work, check manually
GAVRETO didn't work, check manually
SOGROYA didn't work, check manually
ARTESUNATE didn't work, check manually


In [151]:
manual_pulls = []

for drug in manual_check:
    api_params = dict(
        search='openfda.brand_name:' + '\"' + drug + '\"',
        limit=1
    )
    manual_pulls.append(requests.get(url=api_base, params=api_params).json())

In [152]:
api_params = dict(
        search='openfda.brand_name:"COMIRNATY"',
        limit=1
)

requests.get(url=api_base, params=api_params).json()

{'error': {'code': 'NOT_FOUND', 'message': 'No matches found!'}}

## **will need to manually look up info for SOGROYA and ARTESUNATE**

In [153]:
manual_pulls

[{'error': {'code': 'NOT_FOUND', 'message': 'No matches found!'}},
 {'meta': {'disclaimer': 'Do not rely on openFDA to make decisions regarding medical care. While we make every effort to ensure that data is accurate, you should assume all results are unvalidated. We may limit or otherwise restrict your access to the API in line with our Terms of Service.',
   'terms': 'https://open.fda.gov/terms/',
   'license': 'https://open.fda.gov/license/',
   'last_updated': '2022-05-17',
   'results': {'skip': 0, 'limit': 1, 'total': 1}},
  'results': [{'submissions': [{'submission_type': 'SUPPL',
      'submission_number': '5',
      'submission_status': 'AP',
      'submission_status_date': '20220202',
      'review_priority': 'STANDARD',
      'submission_class_code': 'LABELING',
      'submission_class_code_description': 'Labeling',
      'submission_property_type': [{'code': 'Orphan'}],
      'application_docs': [{'id': '70066',
        'url': 'https://www.accessdata.fda.gov/drugsatfda_docs

## Final Columns DrugBank (Selenium)

In [101]:
from selenium import webdriver
import time
from selenium.webdriver.common.keys import Keys

In [102]:
# using port 9999 bc default port was taken, you need to download webdriver that matches current 
# chrome (or other browser) version and replace path if necessary

path = r'C:\Users\chilker\chromedriver.exe'
driver = webdriver.Chrome(path, port=9999)
info = []

driver.get("https://go.drugbank.com/")
driver.implicitly_wait(0.5)

for ind, drug in enumerate(extracted_data['Drug Name']):
    search_box = driver.find_element(by='name', value="query")
    
    search_box.send_keys(drug)
    search_box.send_keys(Keys.ENTER)
    
    if len(driver.current_url) > 40:
        search_box = driver.find_element(by='name', value="query")
        #need to clear search bar, when there's an error website doesn't auto-clear
        search_box.clear()
        
        #try active ingredient when drug name has no hit
        search_box.send_keys(extracted_data['Active Ingredient'][ind])
        search_box.send_keys(Keys.ENTER)
        
        if len(driver.current_url) > 40:
            search_box = driver.find_element(by='name', value="query")
            #need to clear search bar, when there's an error website doesn't auto-clear
            search_box.clear()
        
            info.append((drug, ''))
            continue
        else:
            # don't need to clear in this case
            key = driver.find_elements_by_tag_name('dt')
            val = driver.find_elements_by_tag_name('dd')

            ph_dict = dict(zip([x.text for x in key], [x.text for x in val]))
            info.append((drug, ph_dict))
        continue
        
    else:
        # don't need to clear in this case
        key = driver.find_elements_by_tag_name('dt')
        val = driver.find_elements_by_tag_name('dd')

        ph_dict = dict(zip([x.text for x in key], [x.text for x in val]))
        info.append((drug, ph_dict))
        
    time.sleep(2)
    
    
driver.close()

In [105]:
# errors
[x[0] for x in info if x[1] == '']

['GA 68 PSMA-11',
 'INMAZEB',
 'INQOVI',
 'LYBALVI',
 'NEXTSTELLIS',
 'AZSTARYS',
 'CABENUVA']

In [112]:
drugbank_data = pd.DataFrame()

drugbank_data['Drug Name'] = [x[0] for x in info]
drugbank_data['Type'] = [x[1]['Type'] if x[1] != '' else '' for x in info]
drugbank_data['DrugBank URL'] = (['https://go.drugbank.com/drugs/' + x[1]['DrugBank Accession Number'] 
                                  if x[1] != '' else '' for x in info])

In [186]:
drugbank_data

api_data = api_data.rename(columns={'DrugName': 'Drug Name'})

#extracted_data = extracted_data.drop('No.', axis=1)
extracted_data = extracted_data.replace('GALLIUM 68 PSMA-11', 'GA 68 PSMA-11')

full_bool = full_bool.rename(columns={'Trade Name': 'Drug Name'}).replace(0, False).replace(1, True)

In [187]:
from functools import reduce
data_frames = [drugbank_data, api_data, extracted_data, full_bool]

In [188]:
data_merged = reduce(lambda  left,right: pd.merge(left,right,on=['Drug Name'], how='outer'), data_frames)

In [193]:
data_merged = data_merged.rename(columns={'Drug Name':'DrugName', 'Type':'Type', 'DrugBank URL':'DrugBankURL', 
                            'DosageForm':'DosageForm', 'ApplicationDocsURL':'ApplicationDocsURL', 
                            'ApprovalSponsor':'ApprovalSponsor', 'FilingDate':'FilingDate',
                            'Active Ingredient':'ActiveIngredient', 'Approval Date':'ApprovalDate', 
                            'FDA-approved use on approval date*':'Indication', 
                            'First in Class':'FirstInClass', 'Orphan':'OrphanDrug', 'FT':'FastTrack', 
                            'BT':'FDABreakthrough', 'Priority':'FDAPriority', 
                            'AA':'AcceleratedApproval', 'PDUFA':'MetPDUFAGoal', 
                            '1st Cycle':'FirstCycleApproval', '1st U.S.':'FirstInUS'})

In [197]:
full_fda = pd.concat([past_data, data_merged])

In [198]:
full_fda.to_csv('2011 to 2021 Approvals (no CBER).csv', index=False)

# VACCINES/BIOLOGICS (CBER)

For CBER you have to do DrugBank Selenium pull

For now, just going to manually add CBER, there's only like 15 and it's hard to parse, also some are related to diagnostic/raw materials, not relevant

In [199]:
url_20 = "https://www.fda.gov/vaccines-blood-biologics/development-approval-process-cber/2020-biological-license-application-approvals"
url_21 = "https://www.fda.gov/vaccines-blood-biologics/development-approval-process-cber/2021-biological-license-application-approvals"

In [206]:
pd.read_html(url_20)[0]['Tradename/Proper Name'][0].split('  ')

['TECARTUS', 'brexucabtagene autoleucel']

In [207]:
[x.split('  ') for x in pd.read_html(url_20)[0]['Tradename/Proper Name']]

[['TECARTUS', 'brexucabtagene autoleucel'],
 ['Blood Grouping Reagent, Anti-Lea (Murine Monoclonal)(For Further Manufacturing Use)(FFMU)'],
 ['Blood Grouping Reagent, Anti-Leb (Murine Monoclonal)(For Further Manufacturing Use) (FFMU)'],
 ['MenQuadfi', 'Meningococcal (Groups A, C, Y, W) Conjugate Vaccine'],
 ['SEVENFACT', 'Coagulation Factor VIIa (Recombinant)'],
 ['DG® Gel 8 ABO/Rh (2D)',
  'Blood Grouping Reagent, Anti-D (Monoclonal Blend)'],
 ['AUDENZ', 'Influenza A (H5N1) Monovalent Vaccine, Adjuvanted'],
 ['PALFORZIA Peanut (Arachis hypogaea) Allergen Powder']]

In [214]:
api_base = 'https://api.fda.gov/drug/drugsfda.json?'

api_params = dict(
    search='openfda.brand_name:' + '\"brexucabtagene autoleucel\"', 
    limit=1)

In [215]:
pull = requests.get(url=api_base, params=api_params).json()

In [216]:
pull

{'error': {'code': 'NOT_FOUND', 'message': 'No matches found!'}}

# Appendix

### Other PDF parsing methods

In [ ]:
# import io
# from PyPDF2 import PdfFileReader

In [ ]:
# with open('2021_NewDrugTherapyApprovals_0.pdf', 'rb') as f:
#     pdf = pdftotext.PDF(f)

In [ ]:
# print(pdf[10])

In [ ]:
# response_pdf = requests.get('https://www.fda.gov/media/155227/download')
# pdf_stream = io.BytesIO(response_pdf.content)
# reader = PdfFileReader(pdf_stream)
# page = reader.pages[0]
# print(page.extractText())

In [ ]:
# response_pdf = requests.get('https://www.fda.gov/media/155227/download')
# pdf_stream = io.BytesIO(response_pdf.content)
# reader = PdfFileReader(pdf_stream)

# for page_n in range(len(reader.pages)):
#     page = reader.pages[0]
#     page_text = page.extractText()
    